In [5]:
dataset_id = "college2_staging"

In [6]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'uber-fair:college2_staging' successfully created.


## Loading the CSV files from Cloud Storage

In [13]:
!bq --location=US load --autodetect --skip_leading_rows=1 --source_format=CSV {dataset_id}.Current_Student \
'gs://college_dataset_fmojica/9_15_Current_Students.csv'

Waiting on bqjob_r26ea77a555dd9af9_0000016e5d220d20_1 ... (4s) Current status: DONE   


In [18]:
%%bigquery 
select count(*) from college2_staging.Current_Student

,f0_
0,11


In [16]:
!bq --location=US load --autodetect --skip_leading_rows=1 --source_format=CSV {dataset_id}.New_Students \
'gs://college_dataset_fmojica/9_15_New_Students.csv'

Waiting on bqjob_r4890d7f11a809fc4_0000016e5d232545_1 ... (1s) Current status: DONE   


In [17]:
!bq --location=US load --autodetect --skip_leading_rows=1 --source_format=CSV {dataset_id}.Classes \
'gs://college_dataset_fmojica/9_15_Classes.csv'

Waiting on bqjob_r75996250e54c838c_0000016e5d233285_1 ... (1s) Current status: DONE   


## Creating modeled tables

In [21]:
modeled_dataset_id = "college2_modeled"

In [22]:
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'uber-fair:college2_modeled' successfully created.


In [25]:
%%bigquery
create or replace table college2_modeled.Student as 
select distinct sid, fname, lname, dob
from college2_staging.Current_Student
union distinct
select distinct sid, fname, lname, cast(dob as string) as dob
from college2_staging.New_Students

""


In [26]:
%%bigquery
select count(*) from college2_modeled.Student

,f0_
0,15


In [27]:
%%bigquery
create or replace table college2_modeled.Teacher as 
select distinct tid, instructor, dept
from college2_staging.Classes

""


In [28]:
%%bigquery
select count(*) from college2_modeled.Teacher

,f0_
0,12


In [30]:
%%bigquery 
select * from college2_modeled.Teacher limit 10

,tid,instructor,dept
0,cannata,"Cannata, Philip",CS
1,None,Prof. Glenn Downing,CS
2,None,Prof. Alison Norman,CS
3,mitra,"Mitra, Shyamal",CS
4,cannata,PHILIP CANNATA,CS
5,mueller,"MUELLER, PETER",Math
6,koch,"Koch, Hans",Math
7,tran,"Tran, Ngoc",Mathematics
8,neeman,JOE NEEMAN,Mathematics
9,mitra,"MITRA, SHYAMAL",Computer Science


In [29]:
%%bigquery
create or replace table college2_modeled.Takes as 
select distinct sid, cno, grade
from college2_staging.Current_Student
where sid is not null
and cno is not null

""


In [ ]:
%%bigquery
select * from college2_modeled.Takes limit 10 

In [31]:
%%bigquery
create or replace table college2_modeled.Teaches as 
select distinct tid, cno
from college2_staging.Classes
where tid is not null
and cno is not null

""


In [1]:
%%bigquery
select * from college2_modeled.Teaches limit 10 

,tid,cno
0,koch,M328K
1,tran,M358K
2,bulko,CS313E
3,mitra,CS313E
4,mitra,CS329E
5,neeman,M362K
6,scohen,CS327E
7,cannata,CS347
8,cannata,CS326E
9,mueller,M362K
